I plan on targeting all segments except for segments 3,12, and 27 based on their E[p|x,m] being under the threshold of .002. After multiplying our m_s values by 100 to account for the full size of the segment given that the m_s column makes up only 1% of the segment it belongs to, the total amount of mailers to be sent out is 1,270,800. It will cost $254,160 to run the campaign.

In [1]:
# Import block
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.stats import binom
from math import comb
import scipy.special as sc

In [2]:
def nbetabinomLL(par, data):
    a = par[0]
    b = par[1]
    x = list(data['x_s'])
    m = list(data['m_s'])
    logbeta_ab = (math.lgamma(a) + math.lgamma(b) - math.lgamma(a + b))
    ll_sum = 0
    for idx in range(len(df)):
        ll_sum = ll_sum + (math.lgamma(a + x[idx]) + math.lgamma(b + m[idx] - x[idx]) - math.lgamma(a + b + m[idx]))\
        + math.log(math.comb(m[idx], x[idx])) - logbeta_ab    
    return -ll_sum

def optimizeBB(data):
    bnds = ((0.0001,None),(0.0001,None)) 
    res = minimize(nbetabinomLL, (1,1), (data), method="nelder-mead",
                   options={'xatol': 1e-08, 'disp': True},bounds=bnds)
    print('MLE for a,b is', res.x, 'and LL is',-nbetabinomLL(res.x,data))
    return res

def betabinomial(row):
    ldenom = math.exp(math.lgamma(a)+math.lgamma(b)-math.lgamma(a+b))
    return comb(row["m_s"],row["x_s"]) * math.exp(sc.gammaln([a + row["x_s"]]) + 
                                                  sc.gammaln([b + row["m_s"] - 
                                                              row["x_s"]]) - 
                                                  sc.gammaln([a + b + row["m_s"]]))/ldenom

def expected(row):
    return (a + row['x_s'])/(a + b + row['m_s'])

def full_seg(row):
    return row['m_s'] * 100

In [3]:
#First read in the dataset and assign its contents to a pandas dataframe
df = pd.read_csv('Targeting.csv')

#Takes x over m, outputs it to Test RR
df['Test RR'] = df['x_s']/df['m_s']

#Variables for our specific problem
mailing_cost = 2000/10000
margin = 100
threshold = mailing_cost/margin
total_mailers = sum(df['m_s'])
total_cost = mailing_cost * total_mailers
total_sales = sum(df['x_s'])
profit = total_sales * margin
probability = .1

#Initializing a and b as 1 so that they can be global variables for our program
#and I'll update them after optimization
a = 1
b = 1
b_ab = math.exp(sc.gammaln([a]) + sc.gammaln([b]) - sc.gammaln([a+b]))

<ipython-input-3-86dbde86b5cf>:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b_ab = math.exp(sc.gammaln([a]) + sc.gammaln([b]) - sc.gammaln([a+b]))


In [4]:
print('Threshold: ' , threshold)
print('Probability: ', probability)
print('Mailing Cost: ', mailing_cost)
print('Total Cost:', total_cost)
print('Total Sales: ', total_sales)
print('Profit: ', profit)

Threshold:  0.002
Probability:  0.1
Mailing Cost:  0.2
Total Cost: 2822.0
Total Sales:  62
Profit:  6200


In [5]:
df['Initial Rollout'] = df['Test RR'] > threshold

In [6]:
res = optimizeBB(df)

Optimization terminated successfully.
         Current function value: 73.850394
         Iterations: 104
         Function evaluations: 223
MLE for a,b is [  1.6131702  362.81987103] and LL is -73.85039380829467


In [7]:
#Updating a and b before applying the betabinomial function
a = res.x[0]
b = res.x[1]
a, b

(1.6131702049189092, 362.81987102837655)

In [8]:
#Creating column containing the probability of observing the 
#Test RR in the rest of the segment
df["P(X=x_s|m_s)"] = df.apply(betabinomial, axis=1)

<ipython-input-2-b635ad3835aa>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return comb(row["m_s"],row["x_s"]) * math.exp(sc.gammaln([a + row["x_s"]]) +


In [9]:
#Log Likelihood column
df['LL'] = np.log(df['P(X=x_s|m_s)'])

In [10]:
#Maximized log likelihood
total_LL = sum(df['LL'])
total_LL

-73.85039380828694

In [11]:
#Average response for the whole dataset. Does not
#account for heterogeneity
avg_r_prob = a/(a+b)
avg_r_prob

0.0044265201625508545

In [12]:
#This will end up being our denominator value for the next column
b_ab = math.exp(sc.gammaln([a]) + sc.gammaln([b]) - sc.gammaln([a+b]))
b_ab

<ipython-input-12-6f744a28915c>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b_ab = math.exp(sc.gammaln([a]) + sc.gammaln([b]) - sc.gammaln([a+b]))


6.63814043373229e-05

In [13]:
#Expected probability, includes heterogeneity
df['E[p|x,m]'] = df.apply(expected, axis = 1)

In [14]:
#Column of which segments we'll roll out to given
#our best effort at predicting segment behavior
df['New Rollout'] = df['E[p|x,m]'] > threshold

In [15]:
#New dataframe so as not to mutate the df dataframe
#containing only the segments that we plan to roll
#out to
rslt_df = df.loc[(df['New Rollout'] == True)]

In [16]:
#We only had 1% of the segment's population, so to 
#calculate how many mailers we'll be sending we need
#to multiply that value times 100
rslt_df['mailers'] = df.apply(full_seg, axis = 1)

<ipython-input-16-448e8ae2b262>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['mailers'] = df.apply(full_seg, axis = 1)


In [17]:
rslt_df

,Segment,m_s,x_s,Test RR,Initial Rollout,P(X=x_s|m_s),LL,"E[p|x,m]",New Rollout,mailers
0,1,300,2,0.006667,True,0.163637,-1.810106,0.005438,True,30000
1,2,273,1,0.003663,True,0.280807,-1.270087,0.004100,True,27300
3,4,428,1,0.002336,True,0.248909,-1.390667,0.003298,True,42800
4,5,433,1,0.002309,True,0.247702,-1.395527,0.003277,True,43300
5,6,296,0,0.000000,False,0.382237,-0.961715,0.002443,True,29600
6,7,451,3,0.006652,True,0.117546,-2.140922,0.005657,True,45100
7,8,104,0,0.000000,False,0.666127,-0.406275,0.003444,True,10400
8,9,426,0,0.000000,False,0.285903,-1.252104,0.002041,True,42600
9,10,490,4,0.008163,True,0.080396,-2.520788,0.006569,True,49000
10,11,234,3,0.012821,True,0.068396,-2.682440,0.007709,True,23400


In [18]:
total_mailers = sum(rslt_df['mailers'])
total_mailers

1270800

In [19]:
total_cost = total_mailers * mailing_cost
total_cost

254160.0